# Analysing your results

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

import vstt

### Import

To import an experiment from a psydat file:

In [ ]:
experiment = vstt.Experiment("example.psydat")

### Conditions

The trial conditions are in `trial_list`, each element in this list is a dict of trial conditions that defines a trial:

In [ ]:
experiment.trial_list

This can be more easily viewed if converted to a pandas DataFrame:

In [ ]:
pd.DataFrame(experiment.trial_list)

The `weight` of a trial is how many times it should be repeated.

### Results

The results of the trials are in `stats` which provides both the raw data and calculated statistics as a pandas DataFrame:

In [ ]:
stats = experiment.stats

In [ ]:
stats.head()

- each row in this DataFrame contains the data from a single movement to a target (and optionally back to the center)
- they are in the order that they were shown in the experiment
- `condition_index` is the index of the corresponding trial conditions in `trial_index`

In [ ]:
stats.describe()

### Plot statistics
Here we plot a few of the calculated statics for each target, identifying each set of conditions with a different color:

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(18, 6))
for ax, statistic in zip(
    axs, ["to_target_reaction_time", "to_target_movement_time", "to_target_distance"]
):
    ax.set_title(f"{statistic}")
    for condition_index, df in stats.groupby("condition_index"):
        ax.plot(df.index, df[statistic], label=f"Condition {condition_index}")
    ax.legend()
plt.show()